In [43]:
import numpy as np
from collections import defaultdict
import pandas as pd

In [44]:
dataset_path = "datasets/rating.csv"
n_most_rated_movies = 10000
min_rated_movies = 1
output_file = "storage.npz"

In [45]:
df = pd.read_csv(dataset_path, sep=',', parse_dates=['timestamp'])
df.sort_values('timestamp', inplace=True)
df = df.drop('timestamp', 1)
df.head()

,userId,movieId,rating
4182421,28507,1176,4
18950979,131160,1079,3
18950936,131160,47,5
18950930,131160,21,3
12341178,85252,45,3


In [46]:
movie_count = df['movieId'].value_counts()
most_frequent_movies = movie_count.index.tolist()
np.random.shuffle(most_frequent_movies) # RANDOM MOVIES

movie_ids = most_frequent_movies[:n_most_rated_movies] # 1000 most rated movies

all_movies = set(movie_ids)

all_profiles = defaultdict(list)

for line in df.values:
    uid = int(line[0])
    mid = int(line[1])
    
    if mid not in all_movies:
        continue
        
    rat = float(line[2])
    
    all_profiles[uid].append((mid, rat))
    
print("All profiled loaded")

All profiled loaded


In [47]:
keys = list(all_profiles.keys())
for i in keys:
    if len(all_profiles[i]) < min_rated_movies:
        del all_profiles[i]

print("Number of users: %d" % len(all_profiles.keys()))

Number of users: 138492


In [48]:
ids = list(all_profiles.keys())
np.random.shuffle(ids)

ids = ids[:946]

profiles = defaultdict(list)
tests = defaultdict(list)

for i in ids:
    movies = all_profiles[i]
    limit = int(0.75*len(movies))
    profiles[i] = movies[:limit]
    tests[i] = movies[limit:]

all_movies = list(all_movies)

print("Users and ratings loaded")

Users and ratings loaded


In [49]:
train_size = 0
test_size = 0

for i in profiles.keys():
    train_size += len(profiles[i])
    
for i in tests.keys():
    test_size += len(tests[i])
    
print("Train size: %d" % train_size)
print("Test size: %d" % test_size)

Train size: 36935
Test size: 12784


In [50]:
np.savez(output_file, all_movies=all_movies, profiles=profiles, tests=tests)